# Optimizing Fantasy Football Squad
## Linear Programming with Python PuLP <br /><br />

## Problem Statement
To start playing [Fantasy Premiere League](https://fantasy.premierleague.com/a/squad/selection), we need to select 15 Premiere League football players within a budget of £100m. The players must consist of 2 goalkeepers, 5, defenders, 5 midfielders, and 3 forwarders. Each player has total score that shows how good he is. How would you come up with the best combination of players that will win the game?

**Model the problem** <br />
We can see an optimization problem here to be solved. So we'll model the problem.
<code>
**maximize**: 
total_score_player1 × total_score_player1 + total_score_player2 × player2 + total_score_player3 × player3 + ... + total_score_playerN × playerN
**constraints**:
player1, player2, player3, ..., playerN are either 0 or 1
cost_player1 × player1 + cost_player2 × player2 + cost_player3 × player3 + ... + cost_playerN × playerN <= 100
is_gkp_player1 × player1 + is_gkp_player2 × player2 + is_gkp_player3 × player3 + ... + is_gkp_playerN × playerN = 2
is_def_player1 × player1 + is_def_player2 × player2 + is_def_player3 × player3 + ... + is_def_playerN × playerN = 5
is_mid_player1 × player1 + is_mid_player2 × player2 + is_mid_player3 × player3 + ... + is_mid_playerN × playerN = 5 
is_fwd_player1 × player1 + is_fwd_player2 × player2 + is_fwd_player3 × player3 + ... + is_fwd_playerN × playerN = 3 

</code>

To create a winning squad, I don't want to spend all budget to 15 of the players equally. I want to spend more money to 11 main players and less to the 4 subtitutes. Even further, I want to spend more to some key players within those 11 players. Intuition come into play (wild guess actually), I ended up with these budgeting.

Maximize 5 key players with total £41m

><code>
**maximize**: 
total_score_player1 × total_score_player1 + total_score_player2 × player2 + total_score_player3 × player3 + ... + total_score_playerN × playerN
**constraints**:
player1, player2, player3, ..., playerN are either 0 or 1
cost_player1 × player1 + cost_player2 × player2 + cost_player3 × player3 + ... + cost_playerN × playerN <= 41
is_gkp_player1 × player1 + is_gkp_player2 × player2 + is_gkp_player3 × player3 + ... + is_gkp_playerN × playerN = 1
is_def_player1 × player1 + is_def_player2 × player2 + is_def_player3 × player3 + ... + is_def_playerN × playerN = 1
is_mid_player1 × player1 + is_mid_player2 × player2 + is_mid_player3 × player3 + ... + is_mid_playerN × playerN = 1 
is_fwd_player1 × player1 + is_fwd_player2 × player2 + is_fwd_player3 × player3 + ... + is_fwd_playerN × playerN = 2
</code>

Maximize 6 players with total £39.5m

><code>
**maximize**: 
total_score_player1 × total_score_player1 + total_score_player2 × player2 + total_score_player3 × player3 + ... + total_score_playerN × playerN
**constraints**:
player1, player2, player3, ..., playerN are either 0 or 1
cost_player1 × player1 + cost_player2 × player2 + cost_player3 × player3 + ... + cost_playerN × playerN <= 39.5
is_gkp_player1 × player1 + is_gkp_player2 × player2 + is_gkp_player3 × player3 + ... + is_gkp_playerN × playerN = 0
is_def_player1 × player1 + is_def_player2 × player2 + is_def_player3 × player3 + ... + is_def_playerN × playerN = 3
is_mid_player1 × player1 + is_mid_player2 × player2 + is_mid_player3 × player3 + ... + is_mid_playerN × playerN = 3 
is_fwd_player1 × player1 + is_fwd_player2 × player2 + is_fwd_player3 × player3 + ... + is_fwd_playerN × playerN = 0 
</code>

Maximize 4 subtitutes with total £19.5m

><code>
**maximize**: 
total_score_player1 × total_score_player1 + total_score_player2 × player2 + total_score_player3 × player3 + ... + total_score_playerN × playerN
**constraints**:
player1, player2, player3, ..., playerN are either 0 or 1
cost_player1 × player1 + cost_player2 × player2 + cost_player3 × player3 + ... + cost_playerN × playerN <= 19.5
is_gkp_player1 × player1 + is_gkp_player2 × player2 + is_gkp_player3 × player3 + ... + is_gkp_playerN × playerN = 1
is_def_player1 × player1 + is_def_player2 × player2 + is_def_player3 × player3 + ... + is_def_playerN × playerN = 1
is_mid_player1 × player1 + is_mid_player2 × player2 + is_mid_player3 × player3 + ... + is_mid_playerN × playerN = 1 
is_fwd_player1 × player1 + is_fwd_player2 × player2 + is_fwd_player3 × player3 + ... + is_fwd_playerN × playerN = 1 
</code>

I'll use [Python PuLP](https://www.coin-or.org/PuLP/pulp.html) linear programming to solve the optimization problem and [Python Pandas](https://pandas.pydata.org/) to prepare the data.

In [1]:
import pandas as pd
from pulp import *
import json
import requests
import time

## Prepare the Data
We can find json data for fantasy premiere league [here](https://fantasy.premierleague.com/drf/bootstrap-static). <br />
Pay attention to some important attributes here. *now_cost* represents the player's cost ten times (i.e. £4.5m is 45 here), *total_points* is the total score we are trying to maximize, *element_type* is player's position.

The player's details are stored inside the json node *elements*, load these into a python pandas data frame.

In [17]:
url = 'https://fantasy.premierleague.com/drf/bootstrap-static'
r = ''
while r == '':
    try:
        r = requests.get(url)
    except:
        time.sleep(5)
        continue
r = requests.get(url)
data = json.dumps((r.json()["elements"]))
df = pd.read_json(data, orient = 'records')

In [53]:
#we don't want players that possibly will not play next round.
df = df[(df['chance_of_playing_next_round'] == 100.0)|pd.isnull(df['chance_of_playing_next_round'])].reset_index(drop=True)

In [18]:
#drop unnecessary attributes
df = df[['id','element_type', 'team_code', 'now_cost', 'first_name', 'second_name','total_points', 'form']]

In [19]:
#replace the values in element_type to its respective positions, find it from the original json
df.element_type.replace({1: 'gkp', 2: 'def', 3: 'mid', 4:'fwd'}, inplace = True)

Create four new columns representing each value in element_type. Set the values of the new columns to 1 or 0, element_type:gkp becomes element_type_gkp:1, element_type_def:0, element_type_mid:0, element_type_fwd:0. Pandas has cool function to do the job. 

This is needed so that we can process it with our model.

In [20]:
df = pd.get_dummies(df, columns = ['element_type'])


In [21]:
df = df[df.id!=114] #rule out Diego Costa , He's being transferred but officially still in his team, that's why still in our table

## Model & Solve the Problem
Each player (each row) will be a variable with possible outcome 1 or 0, 1 means chosen. So we have hundreds of variables. We can use PuLP *lpSum* and Sum the columns.

In [22]:
#input dataframe, # of players for each position. return a dataframe of chosen players
def doOptimizePlayers(df, total_cost, n_gkp, n_def, n_mid, n_fwd): 
    idplayers = df.index.tolist()
    prob = LpProblem("fantasy football create squad",LpMaximize)
    players_vars = LpVariable.dicts("plyr", idplayers,0,1,LpInteger)

    prob += lpSum([df.total_points[i]*players_vars[i] for i in idplayers]), "objective: maximize total points"

    prob += lpSum([df.now_cost[i]*players_vars[i] for i in idplayers]) <= total_cost,"cost constrain"

    prob += lpSum([df.element_type_gkp[i]*players_vars[i] for i in idplayers]) == n_gkp, "# goalkeepers"
    prob += lpSum([df.element_type_def[i]*players_vars[i] for i in idplayers]) == n_def, "# defenders"
    prob += lpSum([df.element_type_mid[i]*players_vars[i] for i in idplayers]) == n_mid, "# midfielders"
    prob += lpSum([df.element_type_fwd[i]*players_vars[i] for i in idplayers]) == n_fwd, "# forwarders"   

    #prob.writeLP("FantasyFootball.lp")
    prob.solve()

    values = []
    variables = {x.name: x.varValue for x in prob.variables()}
    for i in players_vars:
        key = players_vars[i]
        values.append(variables[str(key)])
    df['chosen'] = pd.Series(values, index = df.index)

    df_chosen = df[df.chosen > 0].sort_values('total_points', ascending = False)
    return df_chosen

Let's see the five key players! I allocated now_cost 410 for them

In [35]:
df_chosen1 = doOptimizePlayers(df,815,1,4,5,1)
df_chosen1

,id,team_code,now_cost,first_name,second_name,total_points,form,element_type_def,element_type_fwd,element_type_gkp,element_type_mid,chosen
233,234,14,101,Mohamed,Salah,154,8.4,0,0,0,1,1.0
246,247,43,88,Raheem,Sterling,136,8.3,0,0,0,1,1.0
99,100,8,71,Marcos,Alonso,110,6.9,1,0,0,0,1.0
96,97,8,69,César,Azpilicueta,107,4.4,1,0,0,0,1.0
234,235,14,89,Roberto,Firmino,106,7.7,0,1,0,0,1.0
247,248,43,82,David,Silva,105,5.3,0,0,0,1,1.0
244,245,43,63,Nicolás,Otamendi,102,6.6,1,0,0,0,1.0
259,260,1,59,David,de Gea,101,3.9,0,0,1,0,1.0
69,70,36,59,Pascal,Groß,96,3.3,0,0,0,1,1.0
266,267,1,68,Antonio,Valencia,94,3.3,1,0,0,0,1.0


In [36]:
#See if we've used up all 410, we reuse the money if there's some left
print df_chosen1.now_cost.sum()
print df_chosen1.total_points.sum()

814
1203


In [163]:
#We don't want the players we've chosen in the table anymore
df = df[~df.index.isin(df_chosen1.index)]

Let's see the other five main players, I allocated now_cost 395 

In [167]:
df = df[df.id!=159]
df_chosen2  = doOptimizePlayers(df, 138, 1, 1, 0, 1)
print df_chosen2
print df_chosen2.now_cost.sum()
print df_chosen2.total_points.sum()

      id  team_code  now_cost first_name second_name  total_points  form  \
351  352         80        46       Kyle    Naughton            52   3.0   
285  286          4        42     Robert      Elliot            42   1.0   
547  548         11        50      Oumar      Niasse            39   3.8   

     element_type_def  element_type_fwd  element_type_gkp  element_type_mid  \
351                 1                 0                 0                 0   
285                 0                 0                 1                 0   
547                 0                 1                 0                 0   

     chosen  
351     1.0  
285     1.0  
547     1.0  
138
133


In [154]:
df = df[~df.index.isin(df_chosen2.index)]

This time we optimize the four subtitute players with the rest of the money

In [15]:
df_chosen3 = doOptimizePlayers(df, 195, 1, 1, 1, 1)
print df_chosen3
df_chosen3.now_cost.sum()

      id  team_code  now_cost first_name  second_name  total_points  form  \
69    91         90        60        Sam        Vokes           121     0   
17    30         91        45      Artur        Boruc           120     0   
56    77         90        45        Ben          Mee            94     0   
253  320         20        45      Oriol  Romeu Vidal            80     0   

     element_type_def  element_type_fwd  element_type_gkp  element_type_mid  \
69                  0                 1                 0                 0   
17                  0                 0                 1                 0   
56                  1                 0                 0                 0   
253                 0                 0                 0                 1   

     chosen  
69      1.0  
17      1.0  
56      1.0  
253     1.0  


195L

## Final Result!

In [16]:
#combine all results
df_final = pd.concat([df_chosen1,df_chosen2,df_chosen3])[['now_cost', 'first_name', 'second_name','total_points','element_type_def','element_type_fwd','element_type_gkp','element_type_mid', 'team_code']]

In [17]:
#reverse pd.get_dummies element_type into a column position
x = df_final[['element_type_def','element_type_fwd','element_type_gkp','element_type_mid']]
df_final['position'] = x.idxmax(1) 
df_final.drop(axis = 1, labels = ['element_type_def','element_type_fwd','element_type_gkp','element_type_mid']).sort_values(['position','total_points'], ascending = False)

,now_cost,first_name,second_name,total_points,team_code,position
309,95,Bamidele,Alli,225,6,element_type_mid
308,95,Christian,Eriksen,218,6,element_type_mid
354,60,Robert,Snodgrass,133,21,element_type_mid
325,55,Etienne,Capoue,131,57,element_type_mid
253,45,Oriol,Romeu Vidal,80,20,element_type_mid
54,50,Tom,Heaton,149,90,element_type_gkp
17,45,Artur,Boruc,120,91,element_type_gkp
312,125,Harry,Kane,224,6,element_type_fwd
35,75,Joshua,King,178,91,element_type_fwd
69,60,Sam,Vokes,121,90,element_type_fwd


The player looks good! woohoo.

Notes: <br />
* Yeah, playing fantasy football has a lot more to do than this. But it actually helps, lol! And I had fun doing the math & coding, that's the important part.
* Python PuLP might be easy to read if we don't have that many variables. It's not nice to do for loop, iterating the values in the table to feed the PuLP equation. I like [Scipy Linprog's](https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.optimize.linprog.html) syntax better, unfortunately it doesn't support integer linear programming.